In [ ]:
This is the code to split the dataset.


In [ ]:

import os
from sklearn.model_selection import train_test_split
from shutil import copy2, copytree

# Set the path to your training dataset directory
training_dataset_path = "/content/drive/MyDrive/Training"

# Set the path to the directory where you want to store the training and validation sets
output_path = "/content/drive/MyDrive/output"

# Create output directories if they don't exist
train_path = os.path.join(output_path, 'train')
valid_path = os.path.join(output_path, 'validation')
os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)

# List all subdirectories (assuming each subdirectory is a class)
classes = os.listdir(training_dataset_path)

for class_name in classes:
    class_path = os.path.join(training_dataset_path, class_name)
    train_class_path = os.path.join(train_path, class_name)
    valid_class_path = os.path.join(valid_path, class_name)

    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(valid_class_path, exist_ok=True)

    # List all files in the current class directory
    all_files = os.listdir(class_path)

    # Skip split if the number of samples is too small
    if len(all_files) < 2:
        print(f"Skipping split for {class_name} due to insufficient samples.")
        continue

    # Split the files into training and validation sets
    train_files, valid_files = train_test_split(all_files, test_size=0.2, random_state=42)

    # Copy training set files to the 'train' subdirectory for the specific class
    for file in train_files:
        src = os.path.join(class_path, file)
        dst = os.path.join(train_class_path, file)
        if os.path.isdir(src):
            copytree(src, dst)
        else:
            copy2(src, dst)

    # Copy validation set files to the 'validation' subdirectory for the specific class
    for file in valid_files:
        src = os.path.join(class_path, file)
        dst = os.path.join(valid_class_path, file)
        if os.path.isdir(src):
            copytree(src, dst)
        else:
            copy2(src, dst)

print("Training dataset split into training and validation sets for each class.")

Training dataset split into training and validation sets for each class.


This is the code after data splitting, code for the no. of splitted images.


In [ ]:
import os

base_path = "/content/drive/MyDrive/output"
image_types = ["glioma", "meningioma", "notumor", "pituitary"]  # Replace with your actual image types

for image_type in image_types:
    train_folder = os.path.join(base_path, 'train', image_type)
    val_folder = os.path.join(base_path, 'validation', image_type)

    train_count = len(os.listdir(train_folder))
    val_count = len(os.listdir(val_folder))

    print(f"Image type: {image_type}")
    print(f"  Training images: {train_count}")
    print(f"  Validation images: {val_count}")
    print(f"  Total images: {train_count + val_count}\n")

Image type: glioma
  Training images: 1056
  Validation images: 265
  Total images: 1321

Image type: meningioma
  Training images: 1071
  Validation images: 268
  Total images: 1339

Image type: notumor
  Training images: 1276
  Validation images: 319
  Total images: 1595

Image type: pituitary
  Training images: 1165
  Validation images: 292
  Total images: 1457



This is the code for images preprocessing

In [ ]:
import os
import cv2
from tqdm import tqdm

base_path = "/content/drive/MyDrive/output"
image_types = ["glioma", "meningioma", "notumor", "pituitary"]

# Output directory for preprocessed images
preprocessed_path = "/content/drive/MyDrive/processed_images"
os.makedirs(preprocessed_path, exist_ok=True)

# Function to preprocess and save images
def preprocess_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for image_type in image_types:
        input_type_folder = os.path.join(input_folder, image_type)
        output_type_folder = os.path.join(output_folder, image_type)

        os.makedirs(output_type_folder, exist_ok=True)

        # Print the paths for debugging
        print(f"Input folder: {input_type_folder}")
        print(f"Output folder: {output_type_folder}")

        for file_name in tqdm(os.listdir(input_type_folder), desc=f"Processing {image_type}"):
            input_path = os.path.join(input_type_folder, file_name)
            output_path = os.path.join(output_type_folder, file_name)

            # Print the file names for debugging
            print(f"Input file: {input_path}")
            print(f"Output file: {output_path}")

            # Read image
            img = cv2.imread(input_path)

            # Preprocessing steps (e.g., resizing, normalization)
            # Example: Resize to 224x224 pixels
            img = cv2.resize(img, (512, 512))

            # Save preprocessed image
            cv2.imwrite(output_path, img)

# Preprocess training images
preprocess_images(os.path.join(base_path, 'train'), os.path.join(preprocessed_path, 'train'))

# Preprocess validation images
preprocess_images(os.path.join(base_path, 'validation'), os.path.join(preprocessed_path, 'validation'))

In [ ]:
This is the code for create, compile and train the model.(CNN)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set your data paths
train_data_dir = "/content/drive/MyDrive/processed_images/train"
val_data_dir = "/content/drive/MyDrive/processed_images/validation"

# Define the new input shape based on your image size
input_shape = (512, 512, 3)

# Create a simple CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))  # 4 classes (3 tumor types + no tumor)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create data generators for training and validation
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model with validation accuracy monitoring
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    verbose=1  # Set verbose to 1 to see training and validation progress
)

Found 4568 images belonging to 4 classes.
Found 1144 images belonging to 4 classes.
Epoch 1/10
143/143 [==============================] - 3175s 22s/step - loss: 0.9488 - accuracy: 0.7314 - val_loss: 0.3778 - val_accuracy: 0.8794
Epoch 2/10
143/143 [==============================] - 3057s 21s/step - loss: 0.2355 - accuracy: 0.9177 - val_loss: 0.3182 - val_accuracy: 0.8925
Epoch 3/10
143/143 [==============================] - 3058s 21s/step - loss: 0.1096 - accuracy: 0.9634 - val_loss: 0.2801 - val_accuracy: 0.9248
Epoch 4/10
143/143 [==============================] - 3095s 22s/step - loss: 0.0538 - accuracy: 0.9825 - val_loss: 0.2893 - val_accuracy: 0.9309
Epoch 5/10
143/143 [==============================] - 3045s 21s/step - loss: 0.0191 - accuracy: 0.9934 - val_loss: 0.3758 - val_accuracy: 0.9126
Epoch 6/10
143/143 [==============================] - 3073s 21s/step - loss: 0.0325 - accuracy: 0.9908 - val_loss: 0.3674 - val_accuracy: 0.9266
Epoch 7/10
143/143 [==========================

This is the code for the accuracy.

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the 'history' object from model training
# history = model.fit(...)

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


This is the code to save the model.


In [ ]:
 # Save the trained model
 model.save("/content/drive/MyDrive/Brain_Tumor_Model.h5")